# テキスト前処理

In [27]:
import re
from unicodedata import normalize

In [28]:
with open("pdf_to_text/pdf_to_text_0.txt",mode="r",encoding="utf-8") as f:
    text_data = f.read()
text_data = normalize('NFKC', text_data)

In [29]:
text_data[:1000]

'ビジネス光ネットサービス契約約款 \n\n2023年8月1日 \n\n株式会社  STNet \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\x0c\n目    次 \n\n第1章  総  則 \n\n第  1条  約款の適用・・・・・・・・・・・・・・・・・・・・・・・・・・1 \n第  2条  約款の変更・・・・・・・・・・・・・・・・・・・・・・・・・・1 \n第  3条  用語の定義・・・・・・・・・・・・・・・・・・・・・・・・・・1 \n\n第2章  光ネットサービスの種類等 \n\n第  4条  光ネットサービスの種類等・・・・・・・・・・・・・・・・・・・3 \n\n第3章  光ネットサービスの提供区域等 \n\n第  5条  光ネットサービスの提供区域・・・・・・・・・・・・・・・・・・3 \n第  6条  光ネットサービスの提供範囲等・・・・・・・・・・・・・・・・・3 \n\n第4章  契  約 \n\n第  7条  契約の単位・・・・・・・・・・・・・・・・・・・・・・・・・・3 \n第  8条  契約者回線の終端・・・・・・・・・・・・・・・・・・・・・・・3 \n第  9条  収容光ネットサービス取扱局・・・・・・・・・・・・・・・・・・3 \n第10条  光ネット申込みの方法・・・・・・・・・・・・・・・・・・・・・4 \n第11条  光ネット申込みの承諾・・・・・・・・・・・・・・・・・・・・・4 \n第12条  提供開始日・・・・・・・・・・・・・・・・・・・・・・・・・・4 \n第13条  最低利用期間・・・・・・・・・・・・・・・・・・・・・・・・・4 \n第14条  品目及び種別等の変更・・・・・・・・・・・・・・・・・・・・・5 \n第15条  契約者回線等の移転・・・・・・・・・・・・・・・・・・・・・・5 \n第16条  その他の契約内容の変更・・・・・・・・・・・・・・・・・・・・5 \n第17条  利用の一時中断・・・・・・・・・・・・・・・・・・・・・・・・5 \n第18条  光ネットサービス契約に基づく債権・債務の譲渡の禁止・・・・・・5 \n第19条  光ネットサービス契約者が行う光ネットサービス契約の解除・・・・5 \n第20条  当社が

In [30]:
len(text_data.split("\n"))

5552

In [44]:
text_list = [text.replace("\u3000","").replace(" ","") for text in text_data.split("\n") if len(text) > 1]
len(text_list)

3454

In [45]:
text_list[123]

'用語'

In [46]:
with open("pdf_to_text/pdf_to_text_0_1.txt",mode="w",encoding="utf-8") as f:
    f.write("\n".join(text_list))

In [64]:
pattern = "^\d+"
pattern2 = "^第[0-9]+章"
pattern3 = "^第[0-9]+条"
pattern4 = "^\(\D+\)"

In [66]:
text_list2 = []
for text in text_list:
    if re.fullmatch(pattern4,text) or re.fullmatch(pattern,text):
        continue
    elif re.match(pattern3,text):
        text = "\n"+text
    text_list2.append(text)

In [72]:
text_data2 = "\n".join(text_list2)
len(text_data2.split("\n\n"))

126

In [67]:
with open("pdf_to_text/pdf_to_text_0_2.txt",mode="w",encoding="utf-8") as f:
    f.write("\n".join(text_list2))

In [ ]:
with open("pdf_to_text/pdf_to_text_0_2.txt",mode="w",encoding="utf-8") as f:
    f.write("\n".join(text_list2))

## node比較

In [76]:
from llama_index import SimpleDirectoryReader

documents =  SimpleDirectoryReader("pdf_to_text").load_data()
len(documents)

3

In [77]:
paragraph_separator = "\n\n" # 段落分割
chunk_size = 1024 #チャンク（トークン）数
chunk_overlap = 20 # 前のチャンクをどのくらい含めるか
secondary_chunking_regex = '[^,.．;。]+[,.．;。]?'# 文分割

In [78]:
from llama_index.text_splitter import SentenceSplitter
from llama_index.node_parser import SimpleNodeParser
from llama_index import VectorStoreIndex,ServiceContext
import tiktoken

# ノードパーサーの準備
text_splitter = SentenceSplitter(
    paragraph_separator=paragraph_separator,
    secondary_chunking_regex = secondary_chunking_regex,
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
)
node_parser = SimpleNodeParser.from_defaults(
    text_splitter=text_splitter
)
service_context = ServiceContext.from_defaults(
    node_parser=node_parser
)
service_context.node_parser

SimpleNodeParser(text_splitter=SentenceSplitter(chunk_size=1024, chunk_overlap=20, separator=' ', paragraph_separator='\n\n', secondary_chunking_regex='[^,.．;。]+[,.．;。]?', chunking_tokenizer_fn=<function sent_tokenize at 0x7f8c368fcf40>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f8c6ff87e90>, tokenizer=functools.partial(<bound method Encoding.encode of <Encoding 'gpt2'>>, allowed_special='all')), include_metadata=True, include_prev_next_rel=True, metadata_extractor=None, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f8c35bc2650>)

In [94]:
node_list = service_context.node_parser.get_nodes_from_documents([documents[0]])
enc = tiktoken.get_encoding("gpt2")
print(len(node_list))
for node in node_list[10:]:
    print(len(enc.encode(node.text)))
    print(node)
    break

143
830
id_='4502257a-9a44-4d1c-8b91-fc92039b4c55' embedding=None metadata={} excluded_embed_metadata_keys=[] excluded_llm_metadata_keys=[] relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='bbfc2718-398e-48ab-b5ac-aff291ee434c', node_type=None, metadata={}, hash='7796845c2997a442cecac82853c670bc202b233186aed912006f9c94e5fc0ec2'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='fd3091be-d4b6-4edf-a020-a5b4826effbf', node_type=None, metadata={}, hash='592115f5ed09c43d8ec26b27af80fda258dc61c656d1848a4e17c48fcfecaa61'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='77c5828f-4604-43ec-912d-4a3e4045b12c', node_type=None, metadata={}, hash='b875d2a109837c84df71d32cb76da9b62fff297ce830a68b53be4c05d1290976')} hash='4748163ab51b548c09dd7aa1a7c1b78d347a12efe348584f3ed47aa020b1f78c' text='電気通信事業者以外の者が設置する電気通信設備であ\nって、端末設備以外のもの \nその契約者回線の収容される取扱局交換設備が設置さ\nれている光ネットサービス取扱局 \n光ネットサービス契約を継続したまま契約者回線等を\n別の場所に移すこと（同一建物内で端末設備を移転す\nる場合を除きます。） \n端末設備等規則（昭和６０年郵政省令第３１

In [92]:
node_list = service_context.node_parser.get_nodes_from_documents([documents[1]])
enc = tiktoken.get_encoding("gpt2")
print(len(node_list))
for node in node_list[10:]:
    print(len(enc.encode(node.text)))
    print(node.text)
    break

122
980
2当社は、当社の設置した電気通信設備を修理又は復旧する場合のほか、技術上及び光ネ
ットサービスに関する業務の遂行上やむを得ない理由があるときは、収容光ネットサー
ビス取扱局を変更することがあります。
(光ネット申込みの方法)
第10条光ネット申込みをするときは、次に掲げる事項について、当社所定の契約申込書
等に記載し、光ネットサービス取扱所に提出していただきます。
(1)光ネットサービスの品目及び種別等
(2)契約者回線の終端の場所
(3)その他光ネット申込みの内容を特定するための事項
(光ネット申込みの承諾)
第11条光ネットサービス契約は、光ネット申込みに対して当社が承諾したときに成立
します。
2当社は、次のいずれかの場合には、その申込みを承諾しないことがあります。
(1)光ネットサービスを提供することが技術上又は経済上著しく困難なとき。
(2)申込者が光ネットサービスの料金又は工事に関する費用の支払いを現に怠り、又は怠
るおそれがあると当社が判断したとき。
(3)光ネットサービス契約の申込みをした者が、光ネットサービスの利用を停止されてい
る、又は光ネットサービス契約の解除を受けたことがあるとき。
(4)契約申込書に虚偽の事実を記載したことが判明したとき。
(5)第48条(利用に係る契約者の義務)の規定に違反するおそれがあるとき。
(6)別記21に定める提供区域別付加提供条件を満たしていないと当社が判断したとき。
(7)その他光ネットサービスに関する当社の業務の遂行上支障があるとき、又はそのおそ
れがあると当社が判断したとき。


In [93]:
node_list = service_context.node_parser.get_nodes_from_documents([documents[2]])
enc = tiktoken.get_encoding("gpt2")
print(len(node_list))
for node in node_list[10:]:
    print(len(enc.encode(node.text)))
    print(node.text)
    break

116
883
第10条光ネット申込みをするときは、次に掲げる事項について、当社所定の契約申込書
等に記載し、光ネットサービス取扱所に提出していただきます。
(1)光ネットサービスの品目及び種別等
(2)契約者回線の終端の場所
(3)その他光ネット申込みの内容を特定するための事項

第11条光ネットサービス契約は、光ネット申込みに対して当社が承諾したときに成立
します。
2当社は、次のいずれかの場合には、その申込みを承諾しないことがあります。
(1)光ネットサービスを提供することが技術上又は経済上著しく困難なとき。
(2)申込者が光ネットサービスの料金又は工事に関する費用の支払いを現に怠り、又は怠
るおそれがあると当社が判断したとき。
(3)光ネットサービス契約の申込みをした者が、光ネットサービスの利用を停止されてい
る、又は光ネットサービス契約の解除を受けたことがあるとき。
(4)契約申込書に虚偽の事実を記載したことが判明したとき。
(5)第48条(利用に係る契約者の義務)の規定に違反するおそれがあるとき。
(6)別記21に定める提供区域別付加提供条件を満たしていないと当社が判断したとき。
(7)その他光ネットサービスに関する当社の業務の遂行上支障があるとき、又はそのおそ
れがあると当社が判断したとき。

第12条光ネット申込みに基づき、当社が当該光ネットサービスの工事を完了した日を
光ネットサービスの提供を開始した日とします。
